<a href="https://colab.research.google.com/github/Re14m/training/blob/master/2022_04_04_recipie086.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GC mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ディレクトリ移動
%cd /content/drive/'My Drive'/'Colab Notebooks'/'note_Axross'

/content/drive/My Drive/Colab Notebooks/note_Axross


In [ ]:
# ライブラリのインストール

# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from tqdm import tqdm

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras
from keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# dataset DL(https://drive.google.com/open?id=1TfHtRoX73yIJd40QNLEhYzgH_wlhQSLz)
# dataset 解凍
!unrar x tr-music-dataset.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from tr-music-dataset.rar


Would you like to replace the existing file tr-music-dataset/angry/adanali.mp3
710044 bytes, modified on 2018-01-23 10:11
with a new one
710044 bytes, modified on 2018-01-23 10:11

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit a

Extracting  tr-music-dataset/angry/adanali.mp3                             0%  OK 
Extracting  tr-music-dataset/angry/adanli2.mp3                             0%  OK 
Extracting  tr-music-dataset/angry/adini_Feriha_koydum.mp3                 0%  OK 
Extracting  tr-music-dataset/angry/aksiyon_gerilim.mp3                     1%  OK 
Extracting  tr-music-dataset/angry/araf_zamanı_dizi_muzigi.mp3             1%  OK 
Extracting  tr-music-dataset/angry/arka_sokaklar_dizi_muzigi.mp3           1%  OK 
Extracting  tr-music-dataset/angry/arka_sokaklar_dizi_muzigi2.mp3          1%  OK 
Extracting  tr-mus

In [ ]:
# モデル生成（1次元分類、音響的特徴量をdata.csvに変換）
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

moods = 'angry happy relax sad'.split()
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for g in moods:
    for filename in tqdm(os.listdir(f'./tr-music-dataset/{g}')):
        songname = f'./tr-music-dataset/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        # rmse = librosa.feature.rmse(y=y)
        rmse = librosa.feature.rms(y=y) # librosaのバージョンが0.7.0以上の場合はこちらを実行
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

100%|██████████| 100/100 [04:04<00:00,  2.45s/it]


In [ ]:
# 特徴量データの確認
data = pd.read_csv('data.csv', usecols=range(0, 28))
data = data[data['label'].isin(moods)].reset_index(drop=True)
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,turk_savas_muzigi1.mp3,0.3216158449649811,0.096016,2816.623699,2596.022460,5698.115544,0.138925,-109.342041,70.603378,-7.469779,...,3.775610,-9.145604,-1.940450,-9.524535,-5.576449,-17.531982,0.150500,-12.302896,-4.195442,angry
1,adanali.mp3,0.4317035377025604,0.122593,2185.754805,2669.711152,5248.601596,0.064541,-135.374481,87.073929,15.366075,...,5.506364,-0.826706,3.748881,-3.063311,3.716086,-1.213313,3.347859,-1.348656,0.069242,angry
2,aksiyon_gerilim.mp3,0.5116286873817444,0.409924,3096.587660,2958.658588,6727.142655,0.077744,-36.466839,34.828369,-0.812220,...,9.165286,2.176900,9.860042,8.385277,15.627561,8.909006,11.247225,1.037998,3.813335,angry
3,arka_sokaklar_dizi_muzigi.mp3,0.31023791432380676,0.141591,2061.091339,2348.238431,4758.104562,0.085828,-81.503532,104.974533,3.616627,...,3.606392,-5.002441,1.316493,-14.693908,1.772570,-8.380889,0.053923,-13.324963,-2.059277,angry
4,eski_turk_sarkilari.mp3,0.42492058873176575,0.179141,1205.027248,1567.978474,2485.609970,0.047582,-132.302094,159.644058,-25.493837,...,9.108712,4.779320,3.477679,1.929092,-1.830345,-5.184362,5.201424,-1.823227,5.989724,angry


In [ ]:
# 特徴量データのファイルサイズ確認
data.shape

(399, 28)

In [ ]:
# 不要データ列の削除
data = data.drop(['filename'],axis=1)

In [ ]:
# 学習データ・正解データの加工
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [ ]:
# データの正規化
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [ ]:
# データの分離
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# 学習モデルの作成
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
# モデルのコンパイル
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# モデルの保存
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    validation_data=(X_test,y_test),
                    batch_size=20)

Epoch 1/20
16/16 [==============================] - 5s 32ms/step - loss: 1.1432 - accuracy: 0.5423 - val_loss: 0.9513 - val_accuracy: 0.6500
Epoch 2/20
16/16 [==============================] - 0s 6ms/step - loss: 0.7751 - accuracy: 0.6834 - val_loss: 0.7631 - val_accuracy: 0.7250
Epoch 3/20
16/16 [==============================] - 0s 6ms/step - loss: 0.6181 - accuracy: 0.7367 - val_loss: 0.7022 - val_accuracy: 0.7875
Epoch 4/20
16/16 [==============================] - 0s 6ms/step - loss: 0.5273 - accuracy: 0.8025 - val_loss: 0.6508 - val_accuracy: 0.7625
Epoch 5/20
16/16 [==============================] - 0s 6ms/step - loss: 0.4441 - accuracy: 0.8307 - val_loss: 0.6828 - val_accuracy: 0.7625
Epoch 6/20
16/16 [==============================] - 0s 5ms/step - loss: 0.3733 - accuracy: 0.8464 - val_loss: 0.6481 - val_accuracy: 0.7750
Epoch 7/20
16/16 [==============================] - 0s 5ms/step - loss: 0.3268 - accuracy: 0.8966 - val_loss: 0.7144 - val_accuracy: 0.7625
Epoch 8/20
16/16 [=

In [ ]:
# 推論データ(https://drive.google.com/file/d/1hHKyQsjOPDijk89OM_bZwv4MrQQAV7U2/view?usp=sharing)

# 推論データ特徴量書出し
file = open('test.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

filename = 'test.mp3'
y, sr = librosa.load(filename, mono=True, duration=30)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rmse = librosa.feature.rms(y=y)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)
to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'  
for e in mfcc:
    to_append += f' {np.mean(e)}'

file = open('test.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())

In [ ]:
# 推論（モデル利用）
test = pd.read_csv('test.csv')
test = test.drop(['filename'],axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(np.array(test.iloc[:, :-1], dtype = float))

predictions = model.predict(X)
print(predictions)
np.argmax(predictions[0])

[[0.1582438  0.14888263 0.24805176 0.4448218 ]]


3

In [ ]:
# スペクトログラム（値変換による画像化）
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
moods = 'angry happy relax sad'.split()
for g in tqdm(moods):
    i = 0
    os.makedirs(f'img_data/train/{g}', exist_ok=True)
    os.makedirs(f'img_data/test/{g}', exist_ok=True)
    for filename in os.listdir(f'./tr-music-dataset/{g}'):
        i = i + 1
        songname = f'./tr-music-dataset/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        if i % 5 != 0:
            plt.savefig(f'img_data/train/{g}/{filename[:-3].replace(".", "")}.png')
        else:
            plt.savefig(f'img_data/test/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

100%|██████████| 4/4 [06:03<00:00, 90.75s/it]


<Figure size 720x720 with 0 Axes>

In [ ]:
# 学習データの加工
train_datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip = True
)

In [ ]:
# 学習データの加工
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, optimizers
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import load_img,ImageDataGenerator
from keras.callbacks import CSVLogger , ModelCheckpoint

image_size = 224
batch_size = 20

train_dir = 'img_data/train/'
test_dir = 'img_data/test/'

train_datagen = ImageDataGenerator(rescale=1.0 / 255)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=20,
    class_mode='categorical',
    shuffle=True
)

Found 320 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [ ]:
# FineTuning, モデル作成
vgg_conv = VGG16(weights='imagenet', include_top=False,
                 input_shape=(image_size, image_size, 3))
for layer in vgg_conv.layers[:-4]:
    layer.trainable = True

model = models.Sequential()
model.add(vgg_conv)

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.summary()

model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

58900480/58889256 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense_4 (Dense)             (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 21,138,500
Trainable params: 21,138,500
Non-trainable params: 0
___________________________________________

In [ ]:
# 学習の実行とモデルの保存
hist=model.fit_generator(
    train_generator,
    steps_per_epoch=16,
    epochs=20,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=4
)

model.save('music_classification.h5')

Epoch 1/20
16/16 [==============================] - 27s 838ms/step - loss: 1.5672 - accuracy: 0.2719 - val_loss: 1.3685 - val_accuracy: 0.3000
Epoch 2/20
16/16 [==============================] - 13s 768ms/step - loss: 1.4830 - accuracy: 0.2594 - val_loss: 1.3566 - val_accuracy: 0.2500
Epoch 3/20
16/16 [==============================] - 13s 767ms/step - loss: 1.4257 - accuracy: 0.3156 - val_loss: 1.3471 - val_accuracy: 0.2875
Epoch 4/20
16/16 [==============================] - 13s 768ms/step - loss: 1.3623 - accuracy: 0.3125 - val_loss: 1.3153 - val_accuracy: 0.2625
Epoch 5/20
16/16 [==============================] - 13s 769ms/step - loss: 1.3168 - accuracy: 0.3781 - val_loss: 1.2741 - val_accuracy: 0.5500
Epoch 6/20
16/16 [==============================] - 14s 868ms/step - loss: 1.2963 - accuracy: 0.3844 - val_loss: 1.2609 - val_accuracy: 0.3875
Epoch 7/20
16/16 [==============================] - 13s 774ms/step - loss: 1.2139 - accuracy: 0.5125 - val_loss: 1.1719 - val_accuracy: 0.5625

In [ ]:
# 推論

# スペクトログラムファイル作成
cmap = plt.get_cmap('inferno')
filename = 'test.mp3'
plt.figure(figsize=(10,10))
pathlib.Path(f'img_data').mkdir(parents=True, exist_ok=True)     
y, sr = librosa.load(filename, mono=True, duration=5)
plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
plt.axis('off');
plt.savefig(f'img_data/{filename[:-3].replace(".", "")}.png')
plt.clf()

<Figure size 720x720 with 0 Axes>

In [ ]:
# 推論
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array

filename = 'img_data/test.png'
image = load_img(filename, target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)
predictions = model.predict(image)
np.argmax(predictions[0])

3